# Colombian Constitution EDA.

Brief resume about the methodology and the results that we got.

## About the Document

Little research about the context of the document.

## DS Pipeline



## Document Data Model

PowerPoint graph, meanwhile we can use the hierarchy of the articles dictionary

```
jerarquy = {
    'TITULO' : 'h1',
    'DISPOSICIONES' : 'h1',
    'CAPITULO' : 'h2',
    'ARTÍCULO' : 'p'

}
```

## EDA

### 1) Labels and quantity


# Funcion que cuente h1, h2 y articulos

### 2) Number of paragraphs

### 3) Number of words

### 4) Lematizacion y que pasa despues

### 5) Most frequent relevant words

## TFIDF / NMF / LDA